In [1]:
%pip install -qU \
  protobuf>=3.20\
  fastapi\
  uvicorn\
  langchain==0.0.162 \
  openai==0.27.7 \
  tiktoken==0.4.0 \
  "pinecone-client[grpc]"==2.2.1 \
  pinecone_datasets=='0.5.0rc10' \
  openai

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pinecone_datasets=='0.5.0rc10' (from versions: 0.1.5a0, 0.2.2a0, 0.2.3a0, 0.2.4a0, 0.3.1a0, 0.4.0a0, 0.5.0rc1, 0.5.0rc3, 0.5.0rc4, 0.5.0rc5, 0.5.0rc6, 0.5.0rc7, 0.5.0rc8, 0.5.0rc9, 0.5.0rc10, 0.5.0rc11, 0.5.1, 0.6.0, 0.6.1, 0.6.2)
ERROR: No matching distribution found for pinecone_datasets=='0.5.0rc10'

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install pinecone-client

  Obtaining dependency information for pinecone-client from https://files.pythonhosted.org/packages/df/d4/cffbb61236c6c1d7510e835c1ff843e4e7d705ed59d21c0e5b6dc1cb4fd8/pinecone_client-2.2.4-py3-none-any.whl.metadata
  Using cached pinecone_client-2.2.4-py3-none-any.whl.metadata (7.8 kB)
  Obtaining dependency information for loguru>=0.5.0 from https://files.pythonhosted.org/packages/03/0a/4f6fed21aa246c6b49b561ca55facacc2a44b87d65b8b92362a8e99ba202/loguru-0.7.2-py3-none-any.whl.metadata
  Using cached loguru-0.7.2-py3-none-any.whl.metadata (23 kB)
  Obtaining dependency information for dnspython>=2.0.0 from https://files.pythonhosted.org/packages/f6/b4/0a9bee52c50f226a3cbfb54263d02bb421c7f2adc136520729c2c689c1e5/dnspython-2.4.2-py3-none-any.whl.metadata
  Using cached dnspython-2.4.2-py3-none-any.whl.metadata (4.9 kB)
  Using cached win32_setctime-1.1.0-py3-none-any.whl (3.6 kB)
Using cached pinecone_client-2.2.4-py3-none-any.whl (179 kB)
Using cached dnspython-2.4.2-py3-none-any.whl (3


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path="WMT_Grocery_data.csv",source_column='PRODUCT_URL',encoding='utf-8')
dataset = loader.load()
dataset

[Document(page_content='index: 160701\nSHIPPING_LOCATION: 33012\nDEPARTMENT: Baking\nCATEGORY: Flours & Meals\nSUBCATEGORY: \nBREADCRUMBS: Baking/Flours & Meals\nSKU: 10402992\nPRODUCT_URL: https://www.walmart.com/ip/Great-Value-Self-Rising-Flour-5LB-Bag/10402992?fulfillmentIntent=Pickup\nPRODUCT_NAME: Great Value Self-Rising Flour, 5LB Bag\nBRAND: Great Value\nPRICE_RETAIL: 1.96\nPRICE_CURRENT: 1.96\nPRODUCT_SIZE: 5\nPROMOTION: \nRunDate: 2022-09-11 21:20:04\ntid: 16324505', metadata={'source': 'https://www.walmart.com/ip/Great-Value-Self-Rising-Flour-5LB-Bag/10402992?fulfillmentIntent=Pickup', 'row': 0}),
 Document(page_content='index: 8534\nSHIPPING_LOCATION: 72034\nDEPARTMENT: Baking\nCATEGORY: Easy to make\nSUBCATEGORY: \nBREADCRUMBS: Baking/Easy to make\nSKU: 10315215\nPRODUCT_URL: https://www.walmart.com/ip/Great-Value-Sugar-Free-Cherry-Gelatin-Dessert-0-3-oz/10315215?fulfillmentIntent=Pickup\nPRODUCT_NAME: Great Value Sugar Free Cherry Gelatin Dessert, 0.3 oz\nBRAND: Great Valu

In [4]:
len(dataset)

7000

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=860,
    chunk_overlap=240
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(dataset)

In [6]:
docs[5]

Document(page_content='index: 25598\nSHIPPING_LOCATION: 63376\nDEPARTMENT: Snacks\nCATEGORY: Chips\nSUBCATEGORY: \nBREADCRUMBS: Snacks/Chips\nSKU: 31952186\nPRODUCT_URL: https://www.walmart.com/ip/Chex-Mix-Traditional-Savory-Snack-Mix-17-5-oz-Bag/31952186?fulfillmentIntent=Pickup\nPRODUCT_NAME: Chex Mix Traditional Savory Snack Mix, 17.5 oz Bag\nBRAND: Chex Mix\nPRICE_RETAIL: 6.24\nPRICE_CURRENT: 6.24\nPRODUCT_SIZE: 17.5\nPROMOTION: \nRunDate: 2022-09-11 21:20:04\ntid: 16189402', metadata={'source': 'https://www.walmart.com/ip/Chex-Mix-Traditional-Savory-Snack-Mix-17-5-oz-Bag/31952186?fulfillmentIntent=Pickup', 'row': 5})

In [11]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings

#load pinecone api key # find API key in console at app.pinecone.io
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
#load openAI api key
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
# Initialise LLM with required params
llm = ChatOpenAI(temperature=0.1, max_tokens=500,model_name="gpt-3.5-turbo-16k")

In [14]:
# Create the embeddings of the chunks using openAIEmbeddings
embeddings = OpenAIEmbeddings()

In [15]:
index_name = 'walmart-db'

In [20]:
import pinecone

PINECONE_ENVIRONMENT = "asia-southeast1-gcp"
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment='gcp-starter'
)

if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name, # name of the index(Walmart-db)
        metric='cosine',
        dimension=1536,  # 1536 dim of text-embedding-ada-002
    )

In [21]:
# Insert Vector embeddings into pinecone index
from langchain.vectorstores import Pinecone
vectorindex_openai = Pinecone.from_documents(docs, embeddings, index_name = index_name)

In [ ]:
import time
index = pinecone.GRPCIndex(index_name)
# wait a moment for the index to be fully initialized
time.sleep(1)

index.describe_index_stats()